This project builds a Web Scraping Tool that extracts data of a table from a website, transform it into a CSV file and then save it to disc.

The repository Web_Scraping_Tool.git stores all the files of the project.

The target website to scrap data is: https://en.wikipedia.org/wiki/List_of_French_cheeses

In [ ]:
#Create and activate the virtual environment for this notebook (only run it once in the shell).
#python -m venv venv
#source venv/bin/activate
#%pip install -r requirements.txt

In [ ]:
# The website data was previosly downloaded into the file List_of_French_cheeses
# In this cell the data of the website is prepared to be scrapped with the variable reponse.
import scrapy
import os
current_dir = os.path.abspath('')
url = os.path.join(current_dir, "List_of_French_cheeses")
with open(url) as _f:
    url_data = _f.read()

response = scrapy.http.TextResponse(url, body=url_data, encoding='utf-8')

In [ ]:
# First the number of table like objects in the website is determined
len(response.xpath('//table'))

In [ ]:
# All numbers are checked to find the one corresponding to the table we want to scrap.
response.xpath('//table')[0]

In [ ]:
#Once the table is found it is saved in the variable table. The data from the first row printed to check how it is presented. The unnecesary '\n' are noticed. 
table = response.xpath('//table')[0].xpath('tbody')
table.xpath('tr')[37].xpath('td//text()').extract()

In [61]:
print([
        table.xpath('tr')[37].xpath('td[1]/a/text()').extract()[0].replace('\n', ""), 
        table.xpath('tr')[37].xpath('td[2]//text()').extract()[0].replace('\n', ""), 
        table.xpath('tr')[37].xpath('td[3]//text()').extract()[0].replace('\n', ""),
        table.xpath('tr')[37].xpath('td[4]//text()').extract()[0].replace('\n', ""),
        table.xpath('tr')[37].xpath('td[5]//text()').extract()[0].replace('\n', "")])  

['Vacherin', '2006', 'Franche-Comté', 'Cow', 'PDO']


In [62]:
#In this loop all the data is printed with the '\n' removed to check that are the information wanted is available.
for tr in table.xpath('tr'):
    try:
        print([
            tr.xpath('td[1]/a/text()').extract()[0].replace('\n', ""), 
            tr.xpath('td[2]//text()').extract()[0].replace('\n', ""), 
            tr.xpath('td[3]//text()').extract()[0].replace('\n', ""),
            tr.xpath('td[4]//text()').extract()[0].replace('\n', ""),
            tr.xpath('td[5]//text()').extract()[0].replace('\n', "")])
    except:
        pass   

['Abondance', '1990', 'Haute-Savoie', 'Cow', 'PDO']
['Banon', '2003', "Provence-Alpes-Côte d'Azur", 'Goat', 'PDO']
['Beaufort', '1968', 'Savoie', 'Cow', 'PDO']
["Bleu d'Auvergne", '1975', 'Auvergne', 'Cow', 'PDO']
['Bleu des Causses', '1979', 'Midi-Pyrénées', 'Cow', 'PDO']
['Bleu de Gex', '1977', 'Franche-Comté', 'Cow', 'PDO']
['Bleu du Vercors-Sassenage', '1998', 'Rhône-Alpes', 'Cow', 'PDO']
['Brie de Meaux', '1980', 'Île-de-France', 'Cow', 'PDO']
['Brie de Melun', '1980', 'Île-de-France', 'Cow', 'PDO']
['Brillat-Savarin', '2015', 'Burgundy', 'Cow', 'PGI']
['Brocciu', '1983', 'Corsica', 'Sheep', 'PDO']
['Brousse du Rove', '2018', '', '', 'PDO']
['Cabécou', '1988', 'Midi-Pyrénées', 'Goat', 'AOC']
['Cancoillotte', 'n/a', 'Franche-Comté', 'Cow', 'LR']
['Cantal', '1956', 'Auvergne', 'Cow', 'PDO']
['Camembert de Normandie', '1983', 'Normandy', 'Cow', 'AOC']
['Cazelle de Saint Affrique', 'n/a', 'Midi-Pyrénées', 'Sheep', 'PDO']
['Chabichou du Poitou', '1990', 'Poitou-Charentes', 'Goat', 'PDO

In [63]:
# The names of the columns and the data of the rows is saved in lists
column_names = ["Cheese", "Year_of_designated_appellation", "Producing_region", "Type_of_milk", "Designation"]
rows= []
for tr in table.xpath('tr'):
    try:
        Cheese = tr.xpath('td[1]/a/text()').extract()[0].replace('\n', "")
        Year_of_designated_appellation = tr.xpath('td[2]//text()').extract()[0].replace('\n', "")
        Producing_region = tr.xpath('td[3]//text()').extract()[0].replace('\n', "")
        Type_of_milk = tr.xpath('td[4]//text()').extract()[0].replace('\n', "")
        Designation = tr.xpath('td[5]//text()').extract()[0].replace('\n', "")
        rows.append([Cheese, Year_of_designated_appellation, Producing_region, Type_of_milk, Designation])
        
    except:
        pass

print(rows[0][1])
 


1990


In [64]:
# Now persist it to disk
import csv

with open("French_Cheeses.csv", "w") as _f:
    writer = csv.writer(_f)

    # write the column names
    writer.writerow(column_names)

    # now write the rows
    writer.writerows(rows)

In [65]:
# The data is saved in a dictionary whose keys are the name of the cheeses
scrapped_data = {}
for tr in table.xpath('tr'):
    try:
        Cheese = tr.xpath('td[1]/a/text()').extract()[0].replace('\n', "")
        Year_of_designated_appellation = tr.xpath('td[2]//text()').extract()[0].replace('\n', "")
        Producing_region = tr.xpath('td[3]//text()').extract()[0].replace('\n', "")
        Type_of_milk = tr.xpath('td[4]//text()').extract()[0].replace('\n', "")
        Designation = tr.xpath('td[5]//text()').extract()[0].replace('\n', "")
        scrapped_data[Cheese]= [Year_of_designated_appellation, Producing_region, Type_of_milk, Designation]
    except:
        pass
print (scrapped_data['Abondance'])
print (scrapped_data['Abondance'][0])

['1990', 'Haute-Savoie', 'Cow', 'PDO']
1990


In [66]:
# The data is checked before it is saved in a json file.
for key in scrapped_data:
    print (key, scrapped_data[key][0], scrapped_data[key][1], scrapped_data[key][2], scrapped_data[key][3])

Abondance 1990 Haute-Savoie Cow PDO
Banon 2003 Provence-Alpes-Côte d'Azur Goat PDO
Beaufort 1968 Savoie Cow PDO
Bleu d'Auvergne 1975 Auvergne Cow PDO
Bleu des Causses 1979 Midi-Pyrénées Cow PDO
Bleu de Gex 1977 Franche-Comté Cow PDO
Bleu du Vercors-Sassenage 1998 Rhône-Alpes Cow PDO
Brie de Meaux 1980 Île-de-France Cow PDO
Brie de Melun 1980 Île-de-France Cow PDO
Brillat-Savarin 2015 Burgundy Cow PGI
Brocciu 1983 Corsica Sheep PDO
Brousse du Rove 2018   PDO
Cabécou 1988 Midi-Pyrénées Goat AOC
Cancoillotte n/a Franche-Comté Cow LR
Cantal 1956 Auvergne Cow PDO
Camembert de Normandie 1983 Normandy Cow AOC
Cazelle de Saint Affrique n/a Midi-Pyrénées Sheep PDO
Chabichou du Poitou 1990 Poitou-Charentes Goat PDO
Chaource 1970 Champagne-Ardenne Cow PDO
Charolais 2010   PDO
Chevrotin 2003   PDO
Comté 1952 Franche-Comté Cow PDO
Crottin de Chavignol 1976 Centre-Val de Loire Goat PDO
Emmental de Savoie n/a Savoie Cow PGI
Emmental français est-central n/a Franche-Comté Cow PGI
Époisses 1991 Burgund

In [67]:
# The data from the dictionary is stored in a json file
import json
json_data = json.dumps(scrapped_data)

with open("French_Cheeses.json", "w") as _f:

    json.dump(scrapped_data, _f)

In [69]:
# The data is also saved in a sqlite database and different queries are performed
import sqlite3
connection = sqlite3.connect("French_Cheeses")
cursor = connection.cursor()
cursor.execute('CREATE TABLE cheeses (id integer primary key, Cheese text, Year_of_designated_appellation text, Producing_region text, Type_of_milk text, Designation text)')
connection.commit()

In [70]:
query = 'INSERT INTO cheeses (Cheese, Year_of_designated_appellation, Producing_region, Type_of_milk, Designation)VALUES(?,?,?,?,?)'
for tr in table.xpath('tr'):
    try:
        cursor.execute(query , (tr.xpath('td[1]/a/text()').extract()[0].replace('\n', ""), 
                                 tr.xpath('td[2]//text()').extract()[0].replace('\n', ""), 
                                 tr.xpath('td[3]//text()').extract()[0].replace('\n', ""), 
                                 tr.xpath('td[4]//text()').extract()[0].replace('\n', ""), 
                                 tr.xpath('td[5]//text()').extract()[0].replace('\n', "")
                                )) 
        connection.commit()
    except: 
        pass



In [71]:
select_query = 'SELECT COUNT(id) from cheeses'
for i in cursor.execute(select_query):
    print(i)

(63,)


In [72]:
select_query = 'SELECT * from cheeses'
for i in cursor.execute(select_query):
    print(i)

(1, 'Abondance', '1990', 'Haute-Savoie', 'Cow', 'PDO')
(2, 'Banon', '2003', "Provence-Alpes-Côte d'Azur", 'Goat', 'PDO')
(3, 'Beaufort', '1968', 'Savoie', 'Cow', 'PDO')
(4, "Bleu d'Auvergne", '1975', 'Auvergne', 'Cow', 'PDO')
(5, 'Bleu des Causses', '1979', 'Midi-Pyrénées', 'Cow', 'PDO')
(6, 'Bleu de Gex', '1977', 'Franche-Comté', 'Cow', 'PDO')
(7, 'Bleu du Vercors-Sassenage', '1998', 'Rhône-Alpes', 'Cow', 'PDO')
(8, 'Brie de Meaux', '1980', 'Île-de-France', 'Cow', 'PDO')
(9, 'Brie de Melun', '1980', 'Île-de-France', 'Cow', 'PDO')
(10, 'Brillat-Savarin', '2015', 'Burgundy', 'Cow', 'PGI')
(11, 'Brocciu', '1983', 'Corsica', 'Sheep', 'PDO')
(12, 'Brousse du Rove', '2018', '', '', 'PDO')
(13, 'Cabécou', '1988', 'Midi-Pyrénées', 'Goat', 'AOC')
(14, 'Cancoillotte', 'n/a', 'Franche-Comté', 'Cow', 'LR')
(15, 'Cantal', '1956', 'Auvergne', 'Cow', 'PDO')
(16, 'Camembert de Normandie', '1983', 'Normandy', 'Cow', 'AOC')
(17, 'Cazelle de Saint Affrique', 'n/a', 'Midi-Pyrénées', 'Sheep', 'PDO')
(18, 

In [76]:
select_query = 'SELECT * from cheeses WHERE cheese=="Bleu de Gex"'
for i in cursor.execute(select_query):
    print(i)

(6, 'Bleu de Gex', '1977', 'Franche-Comté', 'Cow', 'PDO')


In [77]:
select_query = 'SELECT * from cheeses WHERE Producing_region=="Burgundy"'
for i in cursor.execute(select_query):
    print(i)

(10, 'Brillat-Savarin', '2015', 'Burgundy', 'Cow', 'PGI')
(26, 'Époisses', '1991', 'Burgundy', 'Cow', 'PDO')
(34, 'Mâconnais', '2006', 'Burgundy', 'Goat', 'PDO')
(62, 'Trou du Cru', 'n/a', 'Burgundy', 'Cow', 'AOC')


In [78]:
select_query = 'SELECT * from cheeses WHERE Year_of_designated_appellation > 2010'
for i in cursor.execute(select_query):
    print(i)

(10, 'Brillat-Savarin', '2015', 'Burgundy', 'Cow', 'PGI')
(12, 'Brousse du Rove', '2018', '', '', 'PDO')
(14, 'Cancoillotte', 'n/a', 'Franche-Comté', 'Cow', 'LR')
(17, 'Cazelle de Saint Affrique', 'n/a', 'Midi-Pyrénées', 'Sheep', 'PDO')
(24, 'Emmental de Savoie', 'n/a', 'Savoie', 'Cow', 'PGI')
(25, 'Emmental français est-central', 'n/a', 'Franche-Comté', 'Cow', 'PGI')
(30, 'Gruyère', '2018', 'central eastern regions', 'Cow', 'PGI')
(36, 'Mimolette', 'n/a', 'Nord-Pas-de-Calais', 'Cow', 'LR')
(46, 'Raclette de Savoie', '2015', '', '', 'PGI')
(54, 'Saint-Félicien', 'n/a', 'Rhône-Alpes', 'Cow', 'LR')
(57, 'Soumaintrain', '2014', '', '', 'PGI')
(59, 'Tome fraîche', 'n/a', 'Auvergne', 'Cow', 'AOC')
(60, 'Tomme de Savoie', 'n/a', 'Savoie', 'Cow', 'PGI')
(61, 'Tomme des Pyrénées', 'n/a', 'Midi-Pyrénées', 'Cow', 'PGI')
(62, 'Trou du Cru', 'n/a', 'Burgundy', 'Cow', 'AOC')


In [79]:
#The connection is closed
connection.close()

In [80]:
#The connection is opened once again to perform another query
connection = sqlite3.connect("French_Cheeses")
cursor = connection.cursor()
select_query = 'SELECT * from cheeses WHERE Year_of_designated_appellation > 2010'
for i in cursor.execute(select_query):
    print(i)

(10, 'Brillat-Savarin', '2015', 'Burgundy', 'Cow', 'PGI')
(12, 'Brousse du Rove', '2018', '', '', 'PDO')
(14, 'Cancoillotte', 'n/a', 'Franche-Comté', 'Cow', 'LR')
(17, 'Cazelle de Saint Affrique', 'n/a', 'Midi-Pyrénées', 'Sheep', 'PDO')
(24, 'Emmental de Savoie', 'n/a', 'Savoie', 'Cow', 'PGI')
(25, 'Emmental français est-central', 'n/a', 'Franche-Comté', 'Cow', 'PGI')
(30, 'Gruyère', '2018', 'central eastern regions', 'Cow', 'PGI')
(36, 'Mimolette', 'n/a', 'Nord-Pas-de-Calais', 'Cow', 'LR')
(46, 'Raclette de Savoie', '2015', '', '', 'PGI')
(54, 'Saint-Félicien', 'n/a', 'Rhône-Alpes', 'Cow', 'LR')
(57, 'Soumaintrain', '2014', '', '', 'PGI')
(59, 'Tome fraîche', 'n/a', 'Auvergne', 'Cow', 'AOC')
(60, 'Tomme de Savoie', 'n/a', 'Savoie', 'Cow', 'PGI')
(61, 'Tomme des Pyrénées', 'n/a', 'Midi-Pyrénées', 'Cow', 'PGI')
(62, 'Trou du Cru', 'n/a', 'Burgundy', 'Cow', 'AOC')


In [81]:
#The connection is closed
connection.close()